In [66]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
import import_ipynb #to import lenet from project folder
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam #optimizer 1
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from project.lenet import LeNet
from imutils import paths #install imutils
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2 # install openCV
import os



args= {
    "dataset": Path("images"),
    "model": "hotdog_not_hotdog.model",
    "plot": "plot.png"
    
    
}




# initialize the number of epochs to train for, initial learning rate,
# and batch size
EPOCHS = 25 #how many times you go through a training set
INIT_LR = 1e-3 #learning
BS = 32
 
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []


 
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images(args["dataset"])))
random.seed(42)
random.shuffle(imagePaths)
print(len(imagePaths))

# loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)
 
    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    label = 1 if label == "Hot_dog" else 0
    labels.append(label)
    #print(len(labels))

# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
 
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
    labels, test_size=0.25, random_state=42)
 
# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)


# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")


# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=28, height=28, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS) #optimizer
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"]) #loss function- binary cross entropy because this is 2 class classification problem hot dog not hot dog
 
#train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)
 
# save the model to disk
print("[INFO] serializing network...")
model.save(args["model"])

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Hot_dog/Not Hot_dog")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])







[INFO] loading images...
998
[INFO] compiling model...
[INFO] training network...
Epoch 1/25
23/23 [==============================] - 3s 111ms/step - loss: 0.7150 - acc: 0.4742 - val_loss: 0.6927 - val_acc: 0.5520
Epoch 2/25
23/23 [==============================] - 2s 78ms/step - loss: 0.6935 - acc: 0.5033 - val_loss: 0.6898 - val_acc: 0.5280
Epoch 3/25
23/23 [==============================] - 2s 81ms/step - loss: 0.6897 - acc: 0.5525 - val_loss: 0.6872 - val_acc: 0.5480
Epoch 4/25
23/23 [==============================] - 2s 84ms/step - loss: 0.6899 - acc: 0.5261 - val_loss: 0.6780 - val_acc: 0.6000
Epoch 5/25
23/23 [==============================] - 2s 82ms/step - loss: 0.6821 - acc: 0.5520 - val_loss: 0.6749 - val_acc: 0.6080
Epoch 6/25
23/23 [==============================] - 2s 80ms/step - loss: 0.6840 - acc: 0.5503 - val_loss: 0.6782 - val_acc: 0.5760
Epoch 7/25
23/23 [==============================] - 2s 80ms/step - loss: 0.6858 - acc: 0.5471 - val_loss: 0.6780 - val_acc: 0.6000
